In [7]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import scipy
from scipy import stats
import random
import json
import copy
import math
import pickle
import statistics
import sys
import argparse
import os

# my modules
import scientist
import evaluation
import helper
import settings
import publisher

# global variables
num_bins = 30
num_draws = 10
num_participants = 10
num_generations = 100

bins_to_probs = {}
for i in range(0, num_bins):
    bins_to_probs[i] = np.random.uniform(low=0.0, high=1.0)

In [10]:
def make_participants(setting, alpha_value):
    participants = []

    for i in range (0, num_participants):
        if setting == "rate":
            report_set = settings.ReportingSetting("rate")
        elif setting == "data":
            report_set = settings.ReportingSetting("data")
        elif setting == "subset":
            report_set = settings.ReportingSetting("subset")

        # make participant
        participant = scientist.Participant(alpha=alpha_value, reporting_setting=report_set)
                        
        participants.append(participant)

    return(participants)

def run_experiment_gen_info(setting, alpha_value, rel_pl_data_val, rel_pl_surprise_val, rel_pl_bias_val, scale):
    scientific_record = {}
    for bin_num in range(0, num_bins):
        scientific_record[bin_num] = {} 
        scientific_record[bin_num][0] = 1
        scientific_record[bin_num][1] = 1
    
    kl_per_gen = {}
    
    for generation in range(0, num_generations):       
        # each generation gets an entirely new set of participants
        participants = make_participants(setting, alpha_value)

        # scientists explore and submit reports
        for participant in participants:
            # sample
            for i in range(0, num_draws):
                bin_number, value = participant.sample(scientific_record, num_bins, bins_to_probs)

            # choose the bin
            bin_choice = participant.choose_bin(scientific_record, num_bins, num_draws)

            # make a report
            participant.report(num_bins, num_draws)
            
        # the peer review board selects reports for publication and returns the updated scientific record
        scientific_record = publisher.peer_review_with_scale(participants, scientific_record, rel_pl_data_val, rel_pl_surprise_val, rel_pl_bias_val, num_draws, scale)
        
        kl_per_gen[generation] = evaluation.total_entropy_score(scientific_record, bins_to_probs)
    
    return(evaluation.arm_parameter_score(scientific_record, bins_to_probs), evaluation.total_entropy_score(scientific_record, bins_to_probs), kl_per_gen)


def run_internal_experiment():
    record_scale = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    scale_to_kl_gens = {}
    
    for scale in record_scale:
        print(scale)
        
        average_KL_per_generation = {}
        for i in range(0, num_generations):
            average_KL_per_generation[i] = 0

        for i in range(0, 10):
            print(f"   {i}")
            arm_score, entropy_score, kl_per_gen = run_experiment_gen_info("data", 0, 1, 1, 0, scale)
            for gen_no in kl_per_gen:
                average_KL_per_generation[gen_no] += kl_per_gen[gen_no]

        for key in average_KL_per_generation:
            average_KL_per_generation[key] /= 10
            
        scale_to_kl_gens[scale] = average_KL_per_generation

    # save the results
    pickle.dump(
        scale_to_kl_gens,
        open("/Users/marinamancoridis/Thesis/Thesis_Simulations/scale_to_kl_gens_doubt_new_KL.p", "wb")
    )

In [11]:
run_internal_experiment()

0.0
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.1
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.2
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.3
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.4
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.5
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.6
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.7
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.8
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
0.9
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
1.0
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9


In [16]:
num_generations = 30

def weight_against_kl():
    record_scale = np.linspace(0, 3, 30, endpoint=False)
    
    scale_to_kl_gens = {}
    
    for scale in record_scale:
        print(scale)
        total_score = 0
        scores = []

        for i in range(0, 30):
            print(f"  .{i}")
            arm_score, entropy_score, kl_per_gen = run_experiment_gen_info("data", 0, 1, 1, 0, scale)
            total_score += entropy_score
            scores.append(entropy_score)
            
        scale_to_kl_gens[scale] = (total_score / 30.0, stats.sem(scores))

    # save the results
    pickle.dump(
        scale_to_kl_gens,
        open("/Users/marinamancoridis/Thesis/Thesis_Simulations/record_scale_against_kl_30_gens_new_KL.p", "wb")
    )

In [ ]:
weight_against_kl()

0.0
  .0
  .1
  .2
  .3
  .4
  .5
  .6
  .7
  .8
  .9
  .10
  .11
  .12
  .13
  .14
  .15
  .16
  .17
  .18
  .19
  .20
  .21
  .22
  .23
  .24
  .25
  .26
  .27
  .28
  .29
0.1
  .0
  .1
  .2
  .3
  .4
  .5
  .6
  .7
  .8
  .9
  .10
  .11
  .12
  .13
  .14
  .15
  .16
  .17
  .18
  .19
  .20
  .21
  .22
  .23
  .24
  .25
  .26
  .27
  .28
  .29
0.2
  .0
  .1
  .2
  .3
  .4
  .5
  .6
  .7
  .8
  .9
  .10
  .11
  .12
  .13
  .14
  .15
  .16
  .17
  .18
  .19
  .20
  .21
  .22
  .23
  .24
  .25
  .26
  .27
  .28
  .29
0.30000000000000004
  .0
  .1
  .2
  .3
  .4
  .5
  .6
  .7
  .8
  .9
  .10
  .11
  .12
  .13
  .14
  .15
  .16
  .17
  .18
  .19
  .20
  .21
  .22
  .23
  .24
  .25
  .26
  .27
  .28
  .29
0.4
  .0
  .1
  .2
  .3
  .4
  .5
  .6
  .7
  .8
  .9
  .10
  .11
  .12
  .13
  .14
  .15
  .16
  .17
  .18
  .19
  .20
  .21
  .22
  .23
  .24
  .25
  .26
  .27
  .28
  .29
0.5
  .0
  .1
  .2
  .3
  .4
  .5
  .6
  .7
  .8
  .9
  .10
  .11
  .12
  .13
  .14
  .15
  .16
  .17
  .18
  .19
